In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import pandas as pd

### 1、读取xzyd、zone、road和building

In [25]:
gis_file = 'D:/2Gis/杭州基础数据库.gdb'
xzyd = gpd.read_file(gis_file,layer='L1现状用地xzyd_c4_rk')
zone = gpd.read_file(gis_file,layer='Z1市区交通小区3143')
road = gpd.read_file(gis_file,layer='R1路网中心线_框架数据')
building = gpd.read_file('D:\\2Gis\\Arcgis Packages\\市区建筑\\commondata\\4-gis\\市区建筑.shp')

In [3]:
baidu_population = gpd.read_file('D:\\2Gis\\Arcgis Packages\\OD数据库\\od.gdb', layer='百度常住人口数量202411')
baidu_employment = gpd.read_file('D:\\2Gis\\Arcgis Packages\\OD数据库\\od.gdb', layer='百度就业人口数量202411')

In [4]:
print("Building 原始几何体类型:", building.geom_type.unique())
# 检查是否有任何空几何体（NaN）
print("Building 中空几何体的数量 (NaN):", building.geometry.isna().sum())

Building 原始几何体类型: ['Polygon' 'MultiPolygon']
Building 中空几何体的数量 (NaN): 0


In [26]:
xzyd = xzyd.to_crs("EPSG:4549")
zone = zone.to_crs("EPSG:4549")
road  = road.to_crs("EPSG:4549")
building = building.to_crs("EPSG:4549")
baidu_population = baidu_population.to_crs("EPSG:4549")
baidu_employment = baidu_employment.to_crs("EPSG:4549")

In [ ]:
zone.explore()

### 2、面层间的几个计算函数

In [6]:
# # Analysis for each land-block分析面层内落入的点数量、线长度、面积和等（不分割）
# def analyze_land(land_geometry):
#     # Count features intersecting the neighborhood boundary
#     num_schools = schools[schools.geometry.intersects(neighborhood_geometry)].shape[0]
#     num_subways = subways[subways.geometry.intersects(neighborhood_geometry)].shape[0]
#     bike_path_length = bike_paths[bike_paths.geometry.intersects(neighborhood_geometry)].length.sum()
#     park_area = parks[parks.geometry.intersects(neighborhood_geometry)].area.sum()

#     return num_schools, num_subways, bike_path_length, park_area

In [27]:
def analyze_length(land_geometry):
    # 地块内，道路长度计算
    # 1. 筛选出与地块相交的自行车道（这里的地块只有一块）
    intersecting_road = road[road.geometry.intersects(land_geometry)].copy()    
    road_length = 0
    
    if not intersecting_road.empty:
        # 2. 计算自行车道与邻里几何形状的交集
        # intersection() 方法会返回 LineString 的相交部分
        # (如果 LineString 与 Polygon 相交，结果通常是 LineString 或 MultiLineString)
        intersected_geometries = intersecting_road.geometry.intersection(land_geometry)
        
        # 3. 对相交后的几何图形计算长度并求和
        # 这一步只计算几何图形位于 land_geometry 内部的那一部分的长度
        road_length = intersected_geometries.length.sum()

    # park_area 的计算现在是 jzmj 的折减求和 (沿用您之前的要求)
    # ... (park_area 的计算逻辑与您上一个问题中的答案相同)
    
    return road_length

In [6]:
# 两个面层，根据相交部分面积占比，计算属性和
def calculate_proportional_park_attribute_fast(
    neighborhoods: gpd.GeoDataFrame,
    parks: gpd.GeoDataFrame,
    attribute_col: str = 'BUILDAREA'
):
    if neighborhoods.crs != parks.crs:
        neighborhoods = neighborhoods.to_crs(parks.crs)

    print("Step 1/3: Performing spatial join...")
    neighborhoods_for_join = neighborhoods[['geometry']].copy()
    # ensure a stable, named index
    if neighborhoods_for_join.index.name is None:
        neighborhoods_for_join = neighborhoods_for_join.rename_axis('dk_id')
    right_index_name = neighborhoods_for_join.index.name  # e.g., 'dk_id'

    neighborhoods_for_join = neighborhoods_for_join.rename_geometry('neighborhood_geometry')

    joined_df = gpd.sjoin(
        parks.copy(),
        neighborhoods_for_join,
        how='inner',
        predicate='intersects',
        lsuffix='park',
        rsuffix='neigh',
    )

    # --- Find the right-side id column (can be 'index_right' OR the right index name, e.g. 'dk_id') ---
    candidates = []
    if 'index_right' in joined_df.columns:
        candidates.append('index_right')
    if right_index_name and right_index_name in joined_df.columns:
        candidates.append(right_index_name)
    # common fallback some users see:
    if 'index_neigh' in joined_df.columns:
        candidates.append('index_neigh')

    if not candidates:
        raise KeyError(
            f"Could not find right-side id column after sjoin. "
            f"Available columns: {list(joined_df.columns)}; expected one of "
            f"['index_right', '{right_index_name}', 'index_neigh']"
        )
    right_id_col = candidates[0]  # pick the first match

    # Bring back right-hand geometries aligned by the detected id column
    neigh_geom_aligned = neighborhoods_for_join.loc[
        joined_df[right_id_col], 'neighborhood_geometry'
    ].values
    joined_df = joined_df.reset_index(drop=False)
    joined_df['neighborhood_geometry'] = neigh_geom_aligned

    print("Step 2/3: Calculating intersection area and proportional attribute...")

    joined_df['intersection_geometry'] = joined_df['geometry'].intersection(
        joined_df['neighborhood_geometry']
    )
    joined_df['original_area']   = joined_df['geometry'].area
    joined_df['intersect_area']  = joined_df['intersection_geometry'].area
    joined_df['area_ratio'] = np.where(
        joined_df['original_area'] > 0,
        joined_df['intersect_area'] / joined_df['original_area'],
        0.0
    )
    joined_df['area_ratio'] = np.clip(joined_df['area_ratio'], 0.0, 1.0)
    joined_df['discounted_attribute'] = joined_df[attribute_col] * joined_df['area_ratio']

    print("Step 3/3: Grouping and summing the results...")

    results = (
        joined_df
        .groupby(right_id_col)['discounted_attribute']
        .sum()
        .rename(f'sum_weighted_{attribute_col}')
        .to_frame()
    )

    # Optional: align to all neighborhoods (fill missing with 0)
    results = results.reindex(neighborhoods_for_join.index, fill_value=0.0)

    return results


### 3、计算xzyd图层中building的建筑面积和

In [8]:
# Apply analysis to each neighborhood
zone['road_length'] = zone.geometry.apply(
    lambda geom: pd.Series(analyze_length(geom))
)

In [17]:
# # 运行函数
weighted_jzmj_results = calculate_proportional_park_attribute_fast(xzyd, building, attribute_col='BUILDAREA')
print(weighted_jzmj_results.head())

Step 1/3: Performing spatial join...
Step 2/3: Calculating intersection area and proportional attribute...
Step 3/3: Grouping and summing the results...
       sum_weighted_BUILDAREA
dk_id                        
0                    0.000000
1                    0.000000
2                    0.000000
3                  223.748551
4                   75.739186


### 4、对xzyd和building进行标识，找出没有building的地块，和没有地块的building

In [28]:
# 1) 清理空几何并尝试修复无效几何
xzyd = xzyd[xzyd.geometry.notnull()].copy()
building = building[building.geometry.notnull()].copy()

# 2) 确保两层 CRS 一致
if xzyd.crs != building.crs:
    building = building.to_crs(xzyd.crs)

# 3) 标识出 xzyd 中是否有与任何 building 相交的地块
# 使用 sjoin 左连接 xzyd 到 building，以保留 xzyd 所有记录
j_xz_bd = gpd.sjoin(
    xzyd[['geometry']], building[['geometry']],
    how='left', predicate='intersects'
)
# 找出所有在 sjoin 结果中 'index_right' 不为空（即有相交）的 xzyd 索引
xz_has_bd_idx = j_xz_bd.index[~j_xz_bd['index_right'].isna()].unique()
# 增加字段 'has_building_intersection'
# True 表示有相交，False 表示没有相交
xzyd['has_building_intersection'] = xzyd.index.isin(xz_has_bd_idx)
# 移除筛选子集的代码：xzyd_no_building = xzyd[xzyd['no_building_intersection']].copy()

# 4) 标识出 building 中是否有与任何 xzyd 相交的建筑
# 使用 sjoin 左连接 building 到 xzyd，以保留 building 所有记录
j_bd_xz = gpd.sjoin(
    building[['geometry']], xzyd[['geometry']],
    how='left', predicate='intersects'
)
# 找出所有在 sjoin 结果中 'index_right' 不为空（即有相交）的 building 索引
bd_has_xz_idx = j_bd_xz.index[~j_bd_xz['index_right'].isna()].unique()
# 增加字段 'has_xzyd_intersection'
# True 表示有相交，False 表示没有相交
building['has_xzyd_intersection'] = building.index.isin(bd_has_xz_idx)
# 移除筛选子集的代码：building_no_xzyd = building[building['no_xzyd_intersection']].copy()

# 5) 快速核对数量
# 现在核对的是无相交的数量，它等于 'has_intersection' 字段为 False 的数量
print("无相交地块数量:", len(xzyd[~xzyd['has_building_intersection']]))
print("无相交建筑数量:", len(building[~building['has_xzyd_intersection']]))

# 6) 检查新字段是否已添加到原始 GeoDataFrame 中 (可选)
# print("\nxzyd 的前几行及相交字段：")
# print(xzyd[['has_building_intersection']].head())
# print("\nbuilding 的前几行及相交字段：")
# print(building[['has_xzyd_intersection']].head())

无相交地块数量: 44244
无相交建筑数量: 82377


In [23]:
xzyd.shape

(115795, 63)

In [21]:
building.shape

(1559143, 29)

### 5、计算地块人口、岗位数

#### 5.1根据（1）建筑面积（building和容积率计算，优先前者）、
（2）分行政区人均居住面积（xlsx）（‘现状人口平均面积’、‘现状岗位平均面积’）（‘人口调整系数’、‘岗位调整系数’）计算

In [10]:
xzyd.columns

Index(['BSM', '地块编号', 'xzdkID', 'DLBM', 'DLMC', 'XDLDM', 'XDLMC', 'dlmc_12',
       '现状地类代码', '现状地类名称', '现状主地类代码', '现状主地类名称', '现状主地类编码', 'z_ydf2', '用地类别f',
       'land_type1', 'land_type2', 'LANDUSAGE', 'similarity', 'rjl', 'z_rjl',
       'z_rjl2', 'ldl', 'jzmd', 'jzgd', 'xglx', 'bz', 'ssqx', '供地方式', '权属性质',
       '开发动态', 'intersect_point_count', 'buffer_point_count',
       'total_point_count', 'BUILDAREA', 'allocated_buildarea', 'cal_rjl',
       'ref_rjl2', 'z_rjl3', 'rjl_final', 'jzmj_cal', 'belong_TID',
       'Belong_RegID', 'belong_TIDB', 'belong_jiedao', 'belong_xzq',
       '现状岗位平均面积', '现状人口平均面积', '现状人口数', '现状岗位数', '人口调整系数', '岗位调整系数',
       '现状人口数adj', '现状岗位数adj', '岗位大类', '用地类别f_规划', '岗位大类_规划', 'Shape_Length',
       'Shape_Area', '面积', 'geometry'],
      dtype='object')

In [29]:
xzyd['现状人口数adj'] = np.where(xzyd['allocated_buildarea'].notna(), 
    xzyd['allocated_buildarea'],  # 条件为真时取字段1
    xzyd['jzmj_cal'])/xzyd['现状人口平均面积']*xzyd['人口调整系数']

In [20]:
xzyd['现状人口数'].sum()

np.float64(12223172.821290608)

In [42]:
xzyd['现状人口数adj'].sum()

np.float64(10936914.760426918)

#### 5.2与百度人口点统计进行对比
 (a)排除'用地类别f'中道路、绿地、河流等不可能有居住人口和岗位的用地
（b）用百度人口点与xzyd图层空间进行连接，将所在地块的dkID赋值给点图层
（c）没有相交关系的点，有可能是现状地块有缺失（需要补地块），用building层去连接，并进行标记
（d）与Buidling也没有相交关系的，则找到与其距离最近的地块，将最近地块的dkID赋值给点图层
（e）基于用地类别f进行分组汇总，统计不同区不同用地类别f中的人口、岗位数量


In [ ]:
#  （a）排除 '用地类别f' 中道路、绿地、河流等不可能有居住人口和岗位的用地
#  目前'用地类别f' 中没有这些非建设用地
xzyd['用地类别f'].unique()

array(['R农', 'M', 'A办', 'A中小', 'B办', 'R', 'B', 'A', 'B市', 'RB', 'A文体',
       'A医', 'B旅', 'A教科', 'M创', 'A高', 'R别墅', 'W', None, 'R服', 'R农服'],
      dtype=object)

In [30]:
# (b) 点落到地块（within），赋值 dkID 与 '用地类别f'
join1 = gpd.sjoin(
    baidu_population, xzyd[['xzdkID', '用地类别f', 'geometry']],
    how='left', predicate='within', lsuffix='pop', rsuffix='dk'
)
join1 = join1[~join1.index.duplicated(keep='first')]
join1['match_level'] = np.where(join1['xzdkID'].notna(), 'xzyd', 'unmatched')

In [31]:
# (c) 未匹配点与 building 连接（within）并标记
unmatched_mask = join1['match_level'].eq('unmatched')
if unmatched_mask.any():
    join_bld = gpd.sjoin(
    join1.loc[unmatched_mask, ['geometry']].set_geometry('geometry'),
    building[['OBJECTID', 'LANDUSAGE', 'geometry']], how='left', predicate='within'
    )

    join_bld = join_bld[~join_bld.index.duplicated(keep='first')]
    # 标记落在建筑内的点
    join1.loc[unmatched_mask,'match_level'] = np.where(join_bld['OBJECTID'].notna(), 'building', 'unmatched')  
    join1.loc[unmatched_mask,'用地类别f'] = np.where(join_bld['OBJECTID'].notna(), join_bld['LANDUSAGE'],'')      

In [32]:
# (d) 仍未匹配的点，匹配最近地块（使用有效用地）
still_unmatched = join1['match_level'].eq('unmatched')
if still_unmatched.any():
    near = gpd.sjoin_nearest(
        join1.loc[still_unmatched, ['geometry']].set_geometry('geometry'),
        xzyd[['xzdkID', '用地类别f', 'geometry']],
        how='left', distance_col='nearest_dist'
    )  
    #去重 
    near = near[~near.index.duplicated(keep='first')]
    # 将距离字段合并回原始的 join1（或者使用投影后的 join1_proj 进行后续操作）   

    join1.loc[still_unmatched, 'xzdkID'] = near['xzdkID'].values
    join1.loc[still_unmatched, '用地类别f'] = near['用地类别f'].values
    join1.loc[still_unmatched, 'nearest_dist'] = near['nearest_dist'].values
    join1.loc[still_unmatched, 'match_level'] = 'nearest'
join1

,人数,geometry,index_dk,xzdkID,用地类别f,match_level,nearest_dist
0,174.0,POINT (529543.863 3366715.274),79629.0,79724.0,R,xzyd,NaN
1,15.0,POINT (554174.429 3351174.996),NaN,69015.0,R农,nearest,89.690084
2,200.0,POINT (549808.468 3338360.907),NaN,72495.0,R,nearest,7.898587
3,415.0,POINT (519757.303 3340138.103),112516.0,120036.0,R,xzyd,NaN
4,303.0,POINT (494575.395 3347183.194),83419.0,83514.0,R别墅,xzyd,NaN
...,...,...,...,...,...,...,...
284153,1.0,POINT (531482.054 3355975.001),NaN,18274.0,A中小,nearest,94.687690
284154,1.0,POINT (503032.377 3355688.346),107126.0,109085.0,M,xzyd,NaN
284155,1.0,POINT (531864.744 3343781.64),NaN,61669.0,A办,nearest,63.160361
284156,1.0,POINT (476908.9 3298330.753),NaN,8698.0,M,nearest,7831.678986


In [33]:
baidu_population_merged = baidu_population.join(join1[['xzdkID','用地类别f','match_level','nearest_dist']])

In [28]:
baidu_population_merged

,人数,geometry,xzdkID,用地类别f,match_level,nearest_dist
0,174.0,POINT (529543.863 3366715.274),79724.0,R,xzyd,NaN
1,15.0,POINT (554174.429 3351174.996),69015.0,R农,nearest,89.690084
2,200.0,POINT (549808.468 3338360.907),72495.0,R,nearest,7.898587
3,415.0,POINT (519757.303 3340138.103),120036.0,R,xzyd,NaN
4,303.0,POINT (494575.395 3347183.194),83514.0,R别墅,xzyd,NaN
...,...,...,...,...,...,...
284153,1.0,POINT (531482.054 3355975.001),18274.0,A中小,nearest,94.687690
284154,1.0,POINT (503032.377 3355688.346),109085.0,M,xzyd,NaN
284155,1.0,POINT (531864.744 3343781.64),61669.0,A办,nearest,63.160361
284156,1.0,POINT (476908.9 3298330.753),8698.0,M,nearest,7831.678986


In [34]:
# 根据xzdkID，汇总统计每个地块内baidu人口的总数
xzdk_rk = baidu_population_merged.groupby('xzdkID').agg(
        baidu_population =('人数', 'sum')
    )
xzdk_rk

,baidu_population
xzdkID,
0.0,21.0
2.0,9.0
5.0,1.0
6.0,3.0
7.0,28.0
...,...
126634.0,22.0
126643.0,48.0
126649.0,391.0


In [35]:
xzyd2 = pd.merge(xzyd, xzdk_rk, on='xzdkID', how='left')

In [43]:
xzyd2.columns

Index(['BSM', '地块编号', 'xzdkID', 'DLBM', 'DLMC', 'XDLDM', 'XDLMC', 'dlmc_12',
       '现状地类代码', '现状地类名称', '现状主地类代码', '现状主地类名称', '现状主地类编码', 'z_ydf2', '用地类别f',
       'land_type1', 'land_type2', 'LANDUSAGE', 'similarity', 'rjl', 'z_rjl',
       'z_rjl2', 'ldl', 'jzmd', 'jzgd', 'xglx', 'bz', 'ssqx', '供地方式', '权属性质',
       '开发动态', 'intersect_point_count', 'buffer_point_count',
       'total_point_count', 'BUILDAREA', 'allocated_buildarea', 'cal_rjl',
       'ref_rjl2', 'z_rjl3', 'rjl_final', 'jzmj_cal', 'belong_TID',
       'Belong_RegID', 'belong_TIDB', 'belong_jiedao', 'belong_xzq',
       '现状岗位平均面积', '现状人口平均面积', '现状人口数', '现状岗位数', '人口调整系数', '岗位调整系数',
       '现状人口数adj', '现状岗位数adj', '岗位大类', '用地类别f_规划', '岗位大类_规划', 'Shape_Length',
       'Shape_Area', '面积', 'geometry', 'baidu_population'],
      dtype='object')

#### 5.3分析计算人口和百度汇总人口的差异

In [98]:
baidu_population_merged.to_file(
    'D:\\2Gis\\Arcgis Packages\\OD数据库\\od.gdb', 
    layer='百度常住人口数量202411_tag',
    driver="OpenFileGDB"
)

d:\3BigData\LeaningPython2025\geopandas-coures-files\.venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Page 0 of D:\2Gis\Arcgis Packages\OD数据库\od.gdb\a00000006.freelist contains free area of unexpected size at entry 0
  ogr_write(
d:\3BigData\LeaningPython2025\geopandas-coures-files\.venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Page 3 of D:\2Gis\Arcgis Packages\OD数据库\od.gdb\a00000004.freelist contains free area of unexpected size at entry 3
  ogr_write(
d:\3BigData\LeaningPython2025\geopandas-coures-files\.venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Page 3 of D:\2Gis\Arcgis Packages\OD数据库\od.gdb\a00000004.freelist contains free area of unexpected size at entry 2
  ogr_write(
d:\3BigData\LeaningPython2025\geopandas-coures-files\.venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Page 3 of D:\2Gis\Arcgis Packages\OD数据库\od.gdb\a00000004.freelist contains free area of unexpected size at entry 1
  ogr_write(


### Last、导出gis，保存至gdb

#### 导出前清理z维度和复杂几何类型

In [ ]:
# 将结果合并到原始 xzyd 图层
xzyd_with_attr = xzyd.merge(
    weighted_jzmj_results,
    how='left',         # 保留所有邻里
    left_index=True,    # 使用邻里索引匹配
    right_index=True
)

# 如果某些邻里没有交集，则填 0
xzyd_with_attr['sum_weighted_BUILDAREA'] = xzyd_with_attr['sum_weighted_BUILDAREA'].fillna(0)

# 查看结果
xzyd_with_attr.head()

#### 导出

In [ ]:
# 处理building可能存在的Z属性和极端几何复杂性，确保可导出到gdb
from shapely.geometry import Polygon, MultiPolygon 
# 假设 building 已经加载并完成了相交分析，
# 并且包含了 'has_xzyd_intersection' 字段

# --- 诊断步骤 (运行此部分查看 building 的原始问题) ---

print("--- BUILDING 原始数据诊断 ---")
# 检查 building 的几何体类型 (确认是否只有 Polygon/MultiPolygon)
print("Building 原始几何体类型:", building.geom_type.unique())

# 检查是否有任何空几何体（NaN）
print("Building 中空几何体的数量 (NaN):", building.geometry.isna().sum())

# --- GDB 兼容性修复 ---

# 1. 确保 building 中没有空几何体
building = building[building.geometry.notnull()].copy()

# 2. 定义强制转换函数：将所有几何体转换为 2D MultiPolygon (最安全的方式)
def to_2d_multi_polygon_safe(geom):
    """
    强制将几何体转换为 2D MultiPolygon 类型。
    - 移除 Z 维度
    - 确保 Polygon 被包装成 MultiPolygon
    - 处理 GeometryCollection 等不兼容类型
    """
    if geom is None:
        return None
    
    # a) 强制转换为 2D (移除 Z 坐标)
    # simplify(0) 是移除 Z 维度的常用技巧
    geom_2d = geom.simplify(0)

    # b) 确保类型是 MultiPolygon/Polygon
    if hasattr(geom_2d, 'geoms'):
        # 针对 MultiPolygon, GeometryCollection 等
        polygons = [g for g in geom_2d.geoms if isinstance(g, Polygon) and not g.is_empty]
        if polygons:
            return MultiPolygon(polygons)
        else:
            return None
    elif isinstance(geom_2d, Polygon) and not geom_2d.is_empty:
        # 如果是 Polygon，转换为 MultiPolygon
        return MultiPolygon([geom_2d])
    else:
        # 排除 Point, LineString 或其他无法识别的类型
        return None

# 应用强制转换到 building
building['geometry'] = building['geometry'].apply(to_2d_multi_polygon_safe)

# 3. 清除转换后产生的空行
building = building[building.geometry.notnull()].copy()
print(f"\n清理后 building 剩余记录数: {len(building)}")


# # 4. (额外修复) 将布尔字段转换为整数 (0/1)，避免 GDB 驱动程序对布尔类型的兼容性问题
# if 'has_xzyd_intersection' in building.columns:
#     building['has_xzyd_intersection_int'] = building['has_xzyd_intersection'].astype(int)
#     building = building.drop(columns=['has_xzyd_intersection'])
#     print("已将布尔字段转换为整数类型 (0/1)。")


# # --- 最终导出尝试 ---
# try:
#     building.to_file(
#         gis_file,
#         layer='building_test', # 建议使用新图层名
#         driver="OpenFileGDB"
#     )
#     print("\nBuilding 数据已成功导出到 File GDB。")

# except Exception as e:
#     print(f"\nBuilding 导出最终失败，错误信息: {e}")
#     print("建议：如果仍然失败，请尝试导出到 GeoPackage (.gpkg) 或 Shapefile (.shp)")

In [36]:
xzyd2.to_file(
    gis_file,
    layer='L1现状用地xzyd_test',
    driver="OpenFileGDB"
)

In [ ]:
building.to_file(
    gis_file,
    layer='Building_test',
    driver="OpenFileGDB"
)

In [37]:
# 依据所属街道/区县汇总现状人口与百度人口
agg_cols = ['belong_TID', 'belong_jiedao', 'belong_xzq']
missing_cols = [c for c in agg_cols if c not in xzyd2.columns]
if missing_cols:
    raise KeyError(f"xzyd2 缺少分组列: {missing_cols}")

pop_summary = (
    xzyd2.groupby(agg_cols, dropna=False)[['现状人口数adj', 'baidu_population']]
        .sum(min_count=1)
        .reset_index()
        .rename(columns={
            '现状人口数adj': 'sum_现状人口数adj',
            'baidu_population': 'sum_baidu_population'
        })
)

print('分组汇总完成，结果行数:', len(pop_summary))
try:
    display(pop_summary.head(10))
except Exception:
    print(pop_summary.head(10))


分组汇总完成，结果行数: 3113


,belong_TID,belong_jiedao,belong_xzq,sum_现状人口数adj,sum_baidu_population
0,T00001,高虹镇,临安区,7232.927953,19862.0
1,T00002,高虹镇,临安区,3634.643014,5398.0
2,T00003,高虹镇,临安区,965.044864,3752.0
3,T00004,锦北街道,临安区,941.155857,3503.0
4,T00005,湍口镇,临安区,3361.118986,528910.0
5,T00006,潜川镇,临安区,4142.520694,72197.0
6,T00007,天目山镇,临安区,5666.410655,27152.0
7,T00008,太阳镇,临安区,8468.481849,16170.0
8,T00009,昌化镇,临安区,20835.616953,24020.0
9,T00010,玲珑街道,临安区,4584.790677,14245.0
